In [1]:
from synthetix_v3.base_perps import BasePerps
import matplotlib.pyplot as plt
import polars as pl

In [2]:
base_perps = BasePerps()

In [3]:
settled_orders_df = base_perps.get_settled_orders(timestamp=1696291200) # October 3rd 12:00 AM UTC

In [4]:
polars_df = pl.from_pandas(settled_orders_df)

In [5]:
blacklisted_acct_ids = [1.7014118346046923e+38, # has 1e9 trading vol
                        # 215754283 # has 1e8 trading vol, unsure if whale or an outlier that should be filtered
                        ]

In [6]:
polars_subset_df = polars_df.select([
    pl.col('orderSettleds_timestamp'),
    pl.col('markets_marketSymbol'),
    pl.col('orderSettleds_accountId'),
    pl.col('orderSettleds_fillPrice'),
    pl.col('orderSettleds_accruedFunding'),
    pl.col('orderSettleds_sizeDelta'),
    pl.col('orderSettleds_newSize'),
    pl.col('orderSettleds_totalFees'),
])

In [7]:
preprocessed_df = (polars_subset_df.with_columns([
    pl.from_epoch("orderSettleds_timestamp").alias("datetime"),
    (pl.col('orderSettleds_fillPrice') / 10 ** 18),
    (pl.col('orderSettleds_sizeDelta') / 10 ** 18),
    (pl.col('orderSettleds_accruedFunding') / 10 ** 18),
    (pl.col('orderSettleds_newSize') / 10 ** 18),
    (pl.col('orderSettleds_totalFees') / 10 ** 18),
])
.with_columns([
    (pl.col('orderSettleds_sizeDelta').abs() * pl.col('orderSettleds_fillPrice')).alias('size_usd')
])
.filter(~pl.col('orderSettleds_accountId').is_in(blacklisted_acct_ids))
)

In [8]:
sorted_orders_accts = preprocessed_df.groupby('orderSettleds_accountId').agg([pl.col('size_usd').abs().sum()]).sort(by='size_usd', descending=True)

/tmp/ipykernel_16579/4187212515.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  sorted_orders_accts = preprocessed_df.groupby('orderSettleds_accountId').agg([pl.col('size_usd').abs().sum()]).sort(by='size_usd', descending=True)


In [9]:
# largest accounts amount traded 
for row in sorted_orders_accts.rows(named=False):
    print(f"Account: {row[0]:<13} traded ${row[1]:,.2f}")

Account: 215754283.0   traded $102,055,962.05
Account: 2097112578.0  traded $47,126,328.43
Account: 3968420654.0  traded $28,927,481.67
Account: 2993934055.0  traded $22,149,719.07
Account: 1361418361.0  traded $14,678,728.17
Account: 4000498558.0  traded $14,175,988.08
Account: 2251522513.0  traded $12,113,710.71
Account: 3802029150.0  traded $8,633,029.79
Account: 3202664834.0  traded $7,584,798.00
Account: 2892008584.0  traded $7,508,872.86
Account: 3461391369.0  traded $7,109,674.77
Account: 2557546772.0  traded $6,174,796.77
Account: 2490279701.0  traded $4,779,717.02
Account: 69.0          traded $4,636,973.34
Account: 3341266885.0  traded $4,232,431.88
Account: 20790586.0    traded $3,967,477.04
Account: 1315128020.0  traded $2,990,751.38
Account: 89029.0       traded $2,225,444.61
Account: 3215688029.0  traded $2,040,319.23
Account: 2179405963.0  traded $2,003,973.61
Account: 3143303639.0  traded $1,735,454.87
Account: 91589.0       traded $961,017.05
Account: 2195432464.0  tra

In [10]:
market_oi = preprocessed_df.group_by('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sort(by='size_usd', descending=True)

In [11]:
for row in market_oi.rows(named=False):
    print(f"Market: {row[0]:<5}: OI ${row[1]:,.2f}")

Market: ETH  : OI $268,279,650.87
Market: BTC  : OI $23,995,706.81
Market: OP   : OI $12,229,095.87
Market: SNX  : OI $5,639,228.84
Market: LINK : OI $2,204,675.35


In [12]:
total_trading = preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sum()['size_usd'].item()

/tmp/ipykernel_16579/1252346849.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  total_trading = preprocessed_df.groupby('markets_marketSymbol').agg([pl.col('size_usd').abs().sum()]).sum()['size_usd'].item()


In [13]:
print(f"Total Volume Traded: ${total_trading:,.2f}")

Total Volume Traded: $312,348,357.75
